In [ ]:
from pyomo.environ import *

def setup_model(D, type_mp="LP"):

    TH = len(D)  
    TIME = range(0, TH+1)  
    T = range(1, TH+1)

    if type_mp == "IP":
        type_var = NonNegativeIntegers
    else:
        type_var = NonNegativeReals

    
    m = ConcreteModel()


    m.W = Var(TIME, domain=type_var, bounds=(0, None))
    m.H = Var(TIME, domain=type_var, bounds=(0, None))
    m.L = Var(TIME, domain=type_var, bounds=(0, None))
    m.P = Var(TIME, domain=type_var, bounds=(0, None))
    m.I = Var(TIME, domain=type_var, bounds=(0, None))
    m.S = Var(TIME, domain=type_var, bounds=(0, None))
    m.C = Var(TIME, domain=type_var, bounds=(0, None))
    m.O = Var(TIME, domain=type_var, bounds=(0, None))

    m.Cost = Objective(
        expr=sum(
            640*m.W[t] + 6*m.O[t] + 300*m.H[t] + 500*m.L[t]
            + 2*m.I[t] + 5*m.S[t] + 10*m.P[t] + 30*m.C[t]
            for t in T),
        sense=minimize)
    
    m.labor = Constraint(T,rule = lambda m, t: m.W[t] == m.W[t-1]+m.H[t]-m.L[t])
    m.capacity = Constraint(T,rule = lambda m, t: m.I[t] <= 40*m.W[t]+0.25*m.O[t])
    m.inventory = Constraint(T,rule = lambda m, t: m.I[t] == m.I[t-1]+m.P[t]+m.C[t]-D[t-1]-m.S[t-1]+m.S[t])
    m.overtime = Constraint(T,rule = lambda m, t: m.O[t] <= 10*m.W[t])

    m.W_0 = Constraint(expr=m.W[0] == 80)
    m.I_0 = Constraint(expr=m.I[0] == 1000)
    m.S_0 = Constraint(expr=m.S[0] == 0)

    m.last_inventory = Constraint(expr=m.I[TH] >= 500)
    m.last_shortage = Constraint(expr=m.S[TH] == 0)

    return m

demand=[1600,3000,3200]

model = setup_model(demand,"LP")
model .pprint()


8 Var Declarations
    C : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
    H : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
    I : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : 